# Chapter 3: SARSA
## Author: Wenchang Gao

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import gym

import time
import os

Corridor and TD example

In [ ]:
class Corridor:

  def __init__(self):
    self.board = range(5)
    self.state = 1
    self.observation_space = 5
    self.action_space = 2
  
  def step(self, action):
    state_prime = self.state+1 if action == 0 else self.state-1
    self.state = state_prime
    done = (state_prime==0) or (state_prime==4)
    reward = 1 if state_prime==4 else 0
    return state_prime, reward, done
  
  def reset(self):
    self.state = 1
    return self.state
  
  def render(self):
    for i in range(5):
      print(i if self.state!=i else '*', end=' ')
    print('')


In [ ]:
class TabularSARSA:

  def __init__(self, obs, act, gamma=0.99, epsilon=0.3):
    self.q_table = np.zeros((obs, act), dtype=np.float32)
    self.epsilon = epsilon
    self.gamma = gamma
  
  def print_table(self):
    for i in range(len(self.q_table)):
      for j in range(len(self.q_table[i])):
        print(self.q_table[i][j], end=' ')
      print('')
    print('')

  def act(self, state):
    prob = np.random.random()
    # print(self.act)
    action = np.random.choice(len(self.q_table[state])) \
            if prob<self.epsilon else np.argmax(self.q_table[state])
    return action


Train the agent

In [ ]:
def trainTD(agent=TabularSARSA(5, 2), env=Corridor(), episodes=100):
  for epi in range(episodes):
    display = epi%1000==0
    if display:
      print(f'Episode {epi}:')
    state = env.reset()
    if display:
      env.render()
    action = agent.act(state)
    done = False
    while not done:
      state_prime, reward, done = env.step(action)
      if display:
        env.render()
      action_prime = agent.act(state_prime)
      agent.q_table[state, action] = reward+agent.gamma*agent.q_table[state_prime, action_prime]
      state, action = state_prime, action_prime

  agent.print_table()

In [ ]:
trainTD()

Episode 0:
0 * 2 3 4 
0 1 * 3 4 
0 1 2 * 4 
0 1 * 3 4 
0 * 2 3 4 
0 1 * 3 4 
0 1 2 * 4 
0 1 * 3 4 
0 1 2 * 4 
0 1 2 3 * 
0.0 0.0 
0.98010004 0.0 
0.99 0.0 
1.0 0.0 
0.0 0.0 



SARSA with a neural network to approximate Q-function

SARSA batched memory

In [111]:
class Experience:
  def __init__(self):
    self.label_list = ['states', 'actions', 'rewards', 'next_states',
                       'next_actions', 'dones']
    self.batch = {
        'states':[],
        'actions':[],
        'rewards':[],
        'next_states':[],
        'next_actions':[],
        'dones':[]
    }

  def add_exp(self, exp):
    for i in range(len(self.label_list)):
      self.batch[self.label_list[i]].append(exp[i])

  def reset(self):
    for i in self.label_list:
      self.batch[i] = []

SARSA agent

In [187]:
class Net(nn.Module):
  
  def __init__(self, indim, outdim):
    super(Net, self).__init__()
    self.model = nn.Sequential(*[
        nn.Linear(indim, 32),
        nn.ReLU(),
        nn.Linear(32, 32),
        nn.ReLU(),
        nn.Linear(32, 32),
        nn.ReLU(),
        nn.Linear(32, outdim)
    ])
  
  def forward(self, state):
    return self.model(state)

In [202]:
class SARSA:
  '''
  SARSA based on a neural network
  '''

  def __init__(self, indim, outdim, epsilon=0.9, minie=0.01, gamma=0.999, device='cpu'):
    self.epsilon = epsilon
    self.gamma = gamma
    self.minie = minie
    self.device = device
    self.action_space = outdim
    self.net, self.target = Net(indim, outdim), Net(indim, outdim)
    self.net, self.target = self.net.to(self.device), self.target.to(self.device)
    self.optimizer = optim.Adam(self.net.parameters(), lr=0.01)
    self.training = True
  
  def train(self):
    self.training = True
    self.net.train()
    self.target.train()
  
  def eval(self):
    self.training = False
    self.net.eval()
    self.target.eval()
  
  def epsilon_decay(self):
    if self.epsilon > self.minie: self.epsilon -= 0.001
  
  def forward(self, state):
    return self.net(state)
  
  def egreedy(self, actions):
    prob = np.random.random()
    # print(prob)
    # print(actions)
    return np.random.choice(len(actions)) if prob<self.epsilon else torch.argmax(actions).item()
  
  def learn(self, memory):
    '''
    returns ys according to memory for agent to learn
    '''
    # print(memory.batch)
    # print(memory.batch['states'])
    # states = torch.Tensor(memory.batch['states'], device=self.device)
    # actions = torch.Tensor(memory.batch['actions'], device=self.device)
    # rewards = torch.Tensor(memory.batch['rewards'], device=self.device)
    # next_states, next_actions = torch.Tensor(memory.batch['next_states'], device=self.device), torch.Tensor(memory.batch['next_actions'], device=self.device)
    # dones = torch.Tensor(memory.batch['dones'], device=self.device)
    
    # Nonbatched version
    states = memory.batch['states']
    actions = memory.batch['actions']
    rewards = torch.FloatTensor(memory.batch['rewards'])
    next_states = memory.batch['next_states']
    next_actions = memory.batch['next_actions']
    dones = torch.FloatTensor(memory.batch['dones'])
    N = len(rewards)

    target_q = torch.zeros((N, self.action_space), dtype=torch.float32)
    with torch.no_grad():
      for i in range(len(next_states)):
        # print(next_states[i])
        target_q[i] = self.net(next_states[i])
      target_qa = torch.zeros_like(rewards)
      for i in range(len(rewards)):
        target_qa[i] = target_q[i][next_actions[i]]
      ys = rewards+(1.-dones)*self.gamma*target_qa
    
    qs = torch.zeros_like(target_q)
    for i in range(len(next_states)):
      qs[i] = self.net(states[i])
    y_hats = torch.zeros_like(rewards)
    for i in range(len(rewards)):
      y_hats[i] = qs[i][actions[i]]
    return ys, y_hats

  def act(self, state):
    '''
    choose action based on epsilon-greedy policy
    '''
    actions = self.forward(state)
    if self.training:
      action = self.egreedy(actions)
    else:
      action = torch.argmax(actions).item()
    # print(action)
    return action
  
  def save(self, path):
    torch.save(self.net.state_dict(), path+'.sarsa')
  
  def load(self, path):
    self.net.load_state_dict(path)
    self.net.to(self.device)

In [203]:
def train(env, agent, episodes=8000, lr=0.01):
  memory = Experience()
  criterion = torch.nn.MSELoss()
  loss = 0.
  opt = optim.Adam(agent.net.parameters(), lr=lr)
  for epi in range(episodes):
    agent.train()
    memory.reset()
    state = env.reset()
    state = torch.from_numpy(state)
    action = agent.act(state)
    # print(action)
    done = False
    tot_reward = 0
    while not done:
      next_state, reward, done, _ = env.step(action)
      next_state = torch.from_numpy(next_state)
      tot_reward += reward
      next_action = agent.act(next_state)
      exp = [state, action, reward, next_state, next_action, done]
      memory.add_exp(exp)
      state, action = next_state, next_action
    opt.zero_grad()
    y, y_hat = agent.learn(memory)
    # print(y)
    # print(y_hat)
    loss = criterion(y_hat, y)
    loss.backward()
    opt.step()
    agent.epsilon_decay
    agent.target.load_state_dict(agent.net.state_dict())
    print(f'Episode: {epi}, total reward:{tot_reward}, loss: {loss}')


In [204]:
env = gym.make('CartPole-v0')
indim = env.observation_space.shape[0]
outdim = env.action_space.n 
agent = SARSA(indim, outdim)
# agent.save('CartPolev0')
train(env, agent)

/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:594: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "
/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in futu

Streaming output truncated to the last 5000 lines.
Episode: 3000, total reward:28.0, loss: 9.282913208007812
Episode: 3001, total reward:66.0, loss: 6.467262268066406
Episode: 3002, total reward:13.0, loss: 13.248162269592285
Episode: 3003, total reward:31.0, loss: 7.266739845275879
Episode: 3004, total reward:17.0, loss: 10.563243865966797
Episode: 3005, total reward:20.0, loss: 8.909139633178711
Episode: 3006, total reward:11.0, loss: 4.340092182159424
Episode: 3007, total reward:31.0, loss: 3.4734437465667725
Episode: 3008, total reward:14.0, loss: 4.561675071716309
Episode: 3009, total reward:15.0, loss: 5.796834468841553
Episode: 3010, total reward:13.0, loss: 3.954643726348877
Episode: 3011, total reward:21.0, loss: 6.899512767791748
Episode: 3012, total reward:13.0, loss: 2.3535096645355225
Episode: 3013, total reward:59.0, loss: 7.431161880493164
Episode: 3014, total reward:25.0, loss: 3.4173858165740967
Episode: 3015, total reward:28.0, loss: 5.724348545074463
Episode: 3016, t

In [205]:
agent.eval()
state = env.reset()
state = torch.from_numpy(state)
action = agent.act(state)
done = None
tot_rewards = 0
while not done:
  s_, r, done, _ = env.step(action)
  tot_rewards += r
  state = torch.from_numpy(s_)
  action = agent.act(state)
print(tot_rewards)

200.0


In [ ]:
tokens = 

In [198]:
agent.save('CartPolev0')